In [ ]:
#script to perform dynamic mode decompostion (DMD)
#input file is columns of 1D saturation at different times (see example for clarity)

import numpy as np
from scipy import interpolate
from scipy import fft
from scipy.signal import blackman
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import find_peaks, peak_prominences

#input section 
#################################################
# read saturation profiles from Excel file
swproffilename='31_poro.xlsx'

#read in the desired columns (the sampling time can be varied by final variable)
cols = [i for i in range(1, 100, 1)] 
#################################################

#read in the desired data
swprofiledata = pd.read_excel(swproffilename, usecols=cols, skiprows=[0,1])
dfswprofile=pd.DataFrame(swprofiledata)

######################################################################################
### contour plot 
######################################################################################

xlist   =np.array(dfswprofile.cm)
tlistmax=len(dfswprofile.columns)
tlist   =np.linspace(dfswprofile.columns[2],dfswprofile.columns[tlistmax-1],tlistmax-2)

X, Y = np.meshgrid(xlist, tlist)
Z = np.sqrt(X**2 + Y**2)


for i in range(0,tlistmax-2):
    for ii in range(0,len(dfswprofile[dfswprofile.columns[i+2]])):
        Z[i,ii]=dfswprofile[dfswprofile.columns[i+2]][ii]

levels=np.linspace(0,0.7,81)

fig, ax = plt.subplots(nrows = 1, ncols=1, figsize=(8,5))
cp = ax.contourf(X, Y, Z,levels,cmap='seismic')
clb=fig.colorbar(cp) # Add a colorbar to a plot
clb.set_label('Sw', labelpad=-40, y=1.05, rotation=0)
ax.set_xlabel('x (cm)')
ax.set_ylabel('t (s)')
plt.show()

######################################################################################
#  PyDMD analysis 
######################################################################################

from pydmd import MrDMD
from pydmd import FbDMD
from pydmd import DMD

def make_plot(X, x=None, y=None, figsize=(8, 5), title=''):
    """
    Plot of the data X prior to DMD
    """
    plt.figure(figsize=figsize)
    plt.title(title)
    X = np.real(X)
    CS = plt.pcolor(x, y, X,cmap='seismic')
    cbar = plt.colorbar(CS)
    plt.xlabel('Space')
    plt.ylabel('Time')
    plt.show()
    
data=Z.T    
    
make_plot(data.T, x=xlist, y=tlist)

#dynamic mode decompostion
dmd = DMD(svd_rank=-1, tlsq_rank=0, exact=True, opt=True)  
dmd.fit(X=data)
make_plot(dmd.reconstructed_data.T, x=xlist, y=tlist)

# plot eigenvalues
dmd.plot_eigs(show_axes=True, show_unit_circle=True, figsize=(8, 8))

# print eigenvalues
print('The number of eigenvalues is {}'.format(dmd.eigs.shape[0]))
for eig in dmd.eigs:
    print('Eigenvalue {}: distance from unit circle {}'.format(eig, np.abs(eig.imag**2+eig.real**2 - 1)))


# Plot modes and dynamics
for dynamic in dmd.dynamics:
    plt.plot(tlist, dynamic.real)
    plt.title('Dynamics')
    plt.xlabel('Time')
plt.show()

for mode in dmd.modes.T:
    plt.plot(xlist, mode.real)
    plt.title('Modes')
    plt.xlabel('Space')
plt.show()



In [ ]:
#script to save the modes for the different sampling intervals 
#save in pickle format (this maintains the complex numbers for subsequent analysis)
import pickle

with open("q07_fw05_modes_8.pkl", "wb") as f:
    pickle.dump(dmd.modes.T, f)
    
with open("q07_fw05_Eigenvalues_8.pkl", "wb") as f:
    pickle.dump(dmd.eigs, f)

In [ ]:
#script to plot the number of eigenvalues and their dependence on the sampling interval
from typing import List
import numpy as np
import matplotlib.pyplot as plt

def plot_number_of_eigenvals_times_sum_magnitudes_vs_sampling_interval(fnames: List[str]) -> None:
    """Plots the number of eigenvalues times the sum of their magnitudes vs. the sampling interval for a list of files `fnames`."""

    files = []
    
    #load in files
    for fname in fnames:
        with open(fname, "rb") as f:
            file = pickle.load(f)
            files.append(file)
            
    #split into periodic solution, non-periodic solutions
    def eigenval_stable(data):
        dist_uc = np.abs(data.imag**2+data.real**2)
        ang_x   = np.arctan(data.imag/ data.real) / (2*np.pi)

        figure, axes = plt.subplots()
        cc = plt.Circle(( 0 , 0 ), 1 , fill = False) 
        axes.set_aspect( 1 ) 
        axes.add_artist( cc )
        plt.plot(data.real[(0.95 < dist_uc) & (dist_uc < 1.01)],data.imag[(0.95 < dist_uc) & (dist_uc < 1.01)],'ro')
        plt.plot(data.real[dist_uc <= 0.95],data.imag[dist_uc <= 0.95],'bo')
        plt.ylabel('Imaginary part', fontsize = 12)
        plt.xlabel('Real part', fontsize = 12)
        plt.legend(['Periodic solution', 'Non-periodic solution'])
        plt.xlim(-1.1, 1.1)
        plt.ylim(-1.1, 1.6)

        plt.show()
        periodic = dist_uc[(0.95 < dist_uc) & (dist_uc < 1.01)]
        print("Number of periodic solutions", len(periodic))
        print("Number of non-periodic solutions",  (len(dist_uc) - len(periodic)))
        periodic_list.append(len(periodic))
        non_periodic_list.append((len(dist_uc) - len(periodic)))
        print("Sum of all magnitudes", sum(dist_uc))
        sum_mag.append(sum(dist_uc))
        angle.append(ang_x)
        mag.append(dist_uc)

    periodic_list     = []
    non_periodic_list = []
    sum_mag           = []
    mag               = []
    angle             = []

    for file in files:
        eigenval_stable(file)

    #number of modes for the different sampling intervals
    counts = [len(file) for file in files]
    sampling_int = [1, 2, 4] ## must also be changed when inputs changed 

    print(sum_mag)
    print(counts)
    count_mag = np.array(sum_mag)* np.array(counts)

    logA = np.log(sampling_int)
    logB = np.log(count_mag)

    #logB = np.log(count_list) 

    m, cov      = np.polyfit(logA, logB, 1, cov=True) # fit log(y) = m*log(x) + c
    y_fit       = np.exp(m[0]*logA + m[1]) # calculate the fitted values of y 


    #plt.plot(sampling_int, count_list, color = 'r')
    plt.plot(sampling_int, y_fit, color='red', linestyle='dashed', linewidth = 3)
    plt.plot(sampling_int, count_mag, 'o', markerfacecolor='blue', markersize=10)
    plt.yscale("log")
    plt.xscale("log")
    plt.ylabel('Number of eigenvalues * sum of their magnitude', fontsize = 12)
    plt.xlabel('Sampling interval (s)', fontsize = 12)

    plt.show()
    
  

plot_number_of_eigenvals_times_sum_magnitudes_vs_sampling_interval(["q01_fw07_Eigenvalues_1.pkl", "q01_fw07_Eigenvalues_2.pkl", "q01_fw07_Eigenvalues_4.pkl"])
#enter your file names here to produce figures showing the number of eigenvalues and how they decay with different sampling intervals
#must change sampling int in the script 